In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-defe60e3-353b-9380-7612-e49b58971061)


In [ ]:
from google.colab import drive
from pathlib import Path
import os

ROOT_FOLDER = "generative"
PROJECT_FOLDER = "frame-interpolation"
MOUNTED_PATH = Path('/content/drive')
MYDRIVE_PATH = MOUNTED_PATH / "MyDrive"
PROJECT_PATH = MYDRIVE_PATH /  ROOT_FOLDER / PROJECT_FOLDER
print(PROJECT_PATH)
drive.mount(str(MOUNTED_PATH))

os.makedirs(PROJECT_PATH, exist_ok=True)

/content/drive/MyDrive/generative/frame-interpolation
Mounted at /content/drive


In [ ]:
!git clone https://github.com/google-research/frame-interpolation frame_interpolation
!pip install mediapy==1.0.3 gdown tensorflow==2.8.0

Cloning into 'frame_interpolation'...
remote: Enumerating objects: 241, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 241 (delta 103), reused 52 (delta 52), pack-reused 111
Receiving objects: 100% (241/241), 27.00 MiB | 27.26 MiB/s, done.
Resolving deltas: 100% (135/135), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 112.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 112.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 92.2 MB/s eta 0:00:00
  At

In [ ]:
from pathlib import Path
import functools
import os
from typing import List, Sequence
import sys
import glob
import math
from base64 import b64encode

from IPython import display
from frame_interpolation.eval import interpolator as interpolator_lib
from frame_interpolation.eval import util
from absl import app
from absl import flags
from absl import logging
import mediapy as media
import natsort
import numpy as np
import tensorflow as tf
from tqdm.notebook import trange
from contextlib import contextmanager
import io


In [ ]:
import gdown

models_base_path = PROJECT_PATH / "pretrained_models"

def check_models(model: str) -> bool:
    model_path = models_base_path / f"film_net/{model}/saved_model"

    return (
        (model_path / "keras_metadata.pb").exists() and
        (model_path / "saved_model.pb").exists() and
        (model_path / "variables/variables.data-00000-of-00001").exists() and
        (model_path / "variables/variables.index").exists()
    )


def download_model(url, output_folder):
    return gdown.download_folder(url, output=output_folder, quiet=False, use_cookies=False)

is_downloaded = check_models("L1") and check_models("VGG") and check_models("Style")

gdrive_url = "https://drive.google.com/drive/folders/1q8110-qp225asX3DQvZnfLfJPkCHmDpy"


if not is_downloaded:
    print("Downloading models...")
    result = download_model(gdrive_url, str(models_base_path))
    if result is None:
        msg = f"""\n
=======================================================================
 Automatic download as failed. Please upload following the steps above.
 Upload models to path: {models_base_path}
=======================================================================
        """
        raise UserWarning(msg)
else:
    print("Models already downloaded! Skipping")

Models already downloaded! Skipping


In [ ]:
frames_folder = "interpolate_faces"
times_to_interpolate = 5
transition_pause =  0.3
first_frame_pause =  0.1
video_fps = 30
video_style = "loop"
flim_net_model = "VGG"
INPUT_EXT = ['.png', '.jpg', '.jpeg']

@contextmanager
def capture_stdout():
    # redirect stdout and stderr to a StringIO object
    sys.stdout = sys.stderr = io.StringIO()
    try:
        yield sys.stdout
    except Exception as e:
        # if an exception occurs, print the last 100 lines of output
        output = sys.stdout.getvalue()
        lines = output.strip().split('\n')[-100:]
        print('\n'.join(lines))
        raise e
    finally:
        # reset stdout and stderr to their original values
        sys.stdout = sys.__stdout__
        sys.stderr = sys.__stderr__

if " " in frames_folder:
    msg = f"""

Frames_folder cannot cantain a space in the name.
Please replace the space with _ or -
    """
    raise UserWarning(msg)


In [ ]:
# intput/output directors
frames_folder = Path(frames_folder)
print(frames_folder)
if frames_folder.is_absolute():
    frames_path = frames_folder
    print(f"case1: {frames_path}")
else:
    frames_path = PROJECT_PATH / frames_folder
    print(f"case2 {frames_path}")

intermediate_output = frames_path / 'intermediate_videos'
print(f"intermediate_videos: {intermediate_output}")
# collect input frames
input_files = [str(f)
               for f in map(Path, sorted(glob.glob(f"{str(frames_path)}/*")))
               if f.is_file() and f.suffix in INPUT_EXT]

print(f"input_files: {input_files}")

print("Interpolating", len(input_files), "images")

videos_list_file = f"{frames_path}/videos_files_list.txt"

print(videos_list_file)

!rm "$videos_list_file"
!rm -r "$intermediate_output"
!mkdir -p "$intermediate_output"

if video_style == "loop":
    input_files.append(input_files[0])
elif video_style == "foward_reverse":
    input_files += input_files[1::-1]

frame_sets = list(zip(input_files[:-1], input_files[1:]))

print(f"frame_sets: {frame_sets}")

t_padding_frames = math.floor(video_fps * transition_pause)
s_padding_frames = math.floor(video_fps * first_frame_pause)

print("Loading the model...")
model_path = str(PROJECT_PATH / f"pretrained_models/film_net/{flim_net_model}/saved_model")
interpolator = interpolator_lib.Interpolator(model_path)

for i in trange(len(frame_sets)):
    infiles = frame_sets[i]

    with capture_stdout():
        frames = list(util.interpolate_recursively_from_files(infiles, times_to_interpolate, interpolator))

    # pad the start of the animation if on the first frame
    if i == 0:
        frames = [frames[0]] * s_padding_frames + frames

    frames += [frames[-1]] * t_padding_frames

    # output_frames(frames, str(frames_output_path))
    output_mp4 = f'{intermediate_output}/interpolated_{i:04}.mp4'
    media.write_video(output_mp4, frames, fps=video_fps)


file_namespace = os.path.basename(os.path.normpath(frames_folder)).replace(" ", "_")
final_filename = f"{file_namespace}_{video_fps}FPS_{video_style}.mp4"
print(f'Saving final video to {final_filename}')
all_videos = [f"file '{f}'\n"
              for f in map(Path, sorted(glob.glob(f"{str(intermediate_output)}/*")))
              if f.is_file() and f.suffix == ".mp4"]

with open(videos_list_file, "w") as f:
    f.writelines(all_videos)

final_output_path = f"{frames_path}/{final_filename}"
!ffmpeg -y -f concat -safe 0 -i $videos_list_file -c copy "$final_output_path"


interpolate_faces
case2 /content/drive/MyDrive/generative/frame-interpolation/interpolate_faces
intermediate_videos: /content/drive/MyDrive/generative/frame-interpolation/interpolate_faces/intermediate_videos
input_files: ['/content/drive/MyDrive/generative/frame-interpolation/interpolate_faces/01_frame.png', '/content/drive/MyDrive/generative/frame-interpolation/interpolate_faces/02_frame.png', '/content/drive/MyDrive/generative/frame-interpolation/interpolate_faces/03_frame.png', '/content/drive/MyDrive/generative/frame-interpolation/interpolate_faces/04_frame.png', '/content/drive/MyDrive/generative/frame-interpolation/interpolate_faces/05_frame.png']
Interpolating 5 images
/content/drive/MyDrive/generative/frame-interpolation/interpolate_faces/videos_files_list.txt
rm: cannot remove '/content/drive/MyDrive/generative/frame-interpolation/interpolate_faces/videos_files_list.txt': No such file or directory
frame_sets: [('/content/drive/MyDrive/generative/frame-interpolation/interpolat

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
#@title Play Video

mp4 = open(final_output_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
display.display( display.HTML(f'<video controls loop><source src="{data_url}" type="video/mp4"></video>') )
